In [6]:
from svfpy import svfpy
import numpy as np
import geopandas as gpd
import pandas as pd
import rioxarray

In [7]:
mds = '/media/fernando/DATA/DSM4SVF-sao-paulo.tiff'
svf_file = '/media/fernando/DATA/svf-sp/svf_all.tiff'

In [8]:
svf = svfpy.SVF(
    mds, 
    '/media/fernando/DATA/svf-sp/', 
    kernel_size_side=2500, 
    tmp_folder='/media/fernando/DATA/svf-sp/',
    max_horizontal_distance=10000
)

In [10]:
svf.downscales()

array([ 1.,  1.,  2.,  2.,  4.,  6., 10., 31.])

In [17]:
svf.tangents

array([15.96871942,  5.23874455,  3.03973683,  2.05535637,  1.46986184,
        1.05626819,  0.7174907 ,  0.37118429])

In [4]:
svf_calculated = rioxarray.open_rasterio(svf_file)

In [1]:
#calculo da amostra para 95% de confiança com 5% de margem de erro para
1527000000*4
# 385 amostras
# https://pt.surveymonkey.com/mp/sample-size-calculator/

6108000000

In [6]:
gdf_points = gpd.read_file('validation/sp_validation.gpkg')

In [7]:
t = svf.xmds.rio.transform()
t

Affine(0.5, 0.0, 313330.0,
       0.0, -0.5, 7416216.0)

In [8]:
points = \
gdf_points.geometry.apply(
    lambda x: (int((x.x - t[2]) / t[0]), int((x.y - t[5]) / t[4]))
)

In [9]:
points

0       (48443, 53825)
1      (33141, 109705)
2       (80873, 39703)
3       (45290, 49745)
4       (43137, 54404)
            ...       
380     (34252, 15995)
381    (19709, 107080)
382     (67372, 30140)
383     (39778, 81134)
384     (17040, 59948)
Name: geometry, Length: 385, dtype: object

In [10]:
svf.horizontal_distance

38750.0

In [11]:
svfs, svfs_calculated = [], []

In [12]:
for i, p in enumerate(points):
    print(f'{i}/{len(points)}', end=' ')
    print(p, end=" - ")
    col, row = p
    s = svf.calc_svf_point(col=col, row=row)
    sc = svf_calculated[0, row, col]
    svfs.append(s)
    svfs_calculated.append(int(sc.values))
    print(f'{s} - {sc} - {sc/256}')

0/385 (48443, 53825) - 0.6996557257925821 - 178.0 - 0.6953125
1/385 (33141, 109705) - 0.8724028016971486 - 229.0 - 0.89453125
2/385 (80873, 39703) - 0.5020606252273847 - 187.0 - 0.73046875
3/385 (45290, 49745) - 0.7637470718349456 - 173.0 - 0.67578125
4/385 (43137, 54404) - 0.6630445510456513 - 199.0 - 0.77734375
5/385 (29672, 53620) - 0.14891278934761673 - 80.0 - 0.3125
6/385 (37527, 117202) - 0.877795796009561 - 237.0 - 0.92578125
7/385 (65999, 46606) - 0.8462116277198927 - 236.0 - 0.921875
8/385 (13061, 46779) - 0.789803536909363 - 183.0 - 0.71484375
9/385 (9326, 21153) - 0.8117929291132279 - 223.0 - 0.87109375
10/385 (78586, 34869) - 0.7585575223648311 - 205.0 - 0.80078125
11/385 (12839, 96385) - 0.9226977272261728 - 246.0 - 0.9609375
12/385 (17662, 80109) - 0.9464859589592576 - 261.0 - 1.01953125
13/385 (20046, 103445) - 0.920458845364702 - 250.0 - 0.9765625
14/385 (9675, 68168) - 0.7241571471585418 - 224.0 - 0.875
15/385 (21949, 28683) - 0.3807255994065872 - 177.0 - 0.69140625
16

In [24]:
pd.Series(svfs)

0      0.699656
1      0.872403
2      0.502061
3      0.763747
4      0.663045
         ...   
380    0.789636
381    0.736716
382    0.706293
383    0.821888
384    0.495958
Length: 385, dtype: float64

In [25]:
gdf_points.loc[:, 'svf'] = pd.Series(svfs)

In [26]:
svfs

[0.6996557257925821,
 0.8724028016971486,
 0.5020606252273847,
 0.7637470718349456,
 0.6630445510456513,
 0.14891278934761673,
 0.877795796009561,
 0.8462116277198927,
 0.789803536909363,
 0.8117929291132279,
 0.7585575223648311,
 0.9226977272261728,
 0.9464859589592576,
 0.920458845364702,
 0.7241571471585418,
 0.3807255994065872,
 0.22174411332396415,
 0.8958918776500957,
 0.5562770186382812,
 0.968555992970458,
 0.7949903404806463,
 0.9145925165875827,
 0.554071154343752,
 0.31745423268886636,
 0.9124219453135871,
 0.5283146551222964,
 0.5582855509198643,
 0.35958364969577006,
 0.7398970984651507,
 0.7603313501217391,
 0.6033444901958647,
 0.940750513679171,
 0.8444235843423872,
 0.7470116628160215,
 0.6806214532964381,
 0.47844955317338766,
 0.5339533129557367,
 0.664801176443893,
 0.7061232416641505,
 0.5472190035887902,
 0.9687874191063689,
 0.8028473425096495,
 0.8337791183298053,
 0.8194236816441657,
 0.6449752000833012,
 0.6870906987482253,
 0.8179681833405216,
 0.546802947961

In [27]:
svf_calculated.shape

(1, 145065, 94696)

In [28]:
gdf_points.loc[:, 'svf_calculated'] = pd.Series(svfs_calculated)

In [29]:
gdf_points.loc[:, 'svf_calculated_perc'] = pd.Series(svfs_calculated)/256

In [30]:
gdf_points.to_file("validation/sp_patial_validation.gpkg", driver='GPKG')

In [31]:
gdf_points.corr()

,rand_point_id,svf,svf_calculated,svf_calculated_perc
rand_point_id,1.000000,0.066747,0.082650,0.082650
svf,0.066747,1.000000,0.882071,0.882071
svf_calculated,0.082650,0.882071,1.000000,1.000000
svf_calculated_perc,0.082650,0.882071,1.000000,1.000000
